## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,60.91,76,100,6.40,overcast clouds
1,1,Pevek,RU,69.7008,170.3133,35.51,91,34,6.29,scattered clouds
2,2,Luderitz,NaN,-26.6481,15.1594,59.76,81,1,3.06,clear sky
3,3,Hithadhoo,MV,-0.6000,73.0833,82.76,73,0,9.78,clear sky
4,4,Albany,US,42.6001,-73.9662,87.75,80,100,1.99,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hithadhoo,MV,-0.6000,73.0833,82.76,73,0,9.78,clear sky
4,4,Albany,US,42.6001,-73.9662,87.75,80,100,1.99,moderate rain
6,6,Buala,SB,-8.1450,159.5921,80.47,80,92,10.27,overcast clouds
9,9,Ontario,US,34.0633,-117.6509,80.26,51,20,4.61,few clouds
18,18,Kapaa,US,22.0752,-159.3190,80.92,73,75,5.01,light rain
19,19,Padang,ID,-0.9492,100.3543,77.23,87,100,3.18,overcast clouds
25,25,Atuona,PF,-9.8000,-139.0333,79.20,80,85,17.40,light rain
27,27,Hilo,US,19.7297,-155.0900,79.79,77,90,5.75,light rain
31,31,Victoria,HK,22.2855,114.1577,82.45,86,100,1.01,overcast clouds
35,35,Korla,CN,41.7597,86.1469,78.73,27,45,3.76,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
#len(preferred_cities_df)

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
len(clean_df)

204

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hithadhoo,MV,82.76,clear sky,-0.6000,73.0833,
4,Albany,US,87.75,moderate rain,42.6001,-73.9662,
6,Buala,SB,80.47,overcast clouds,-8.1450,159.5921,
9,Ontario,US,80.26,few clouds,34.0633,-117.6509,
18,Kapaa,US,80.92,light rain,22.0752,-159.3190,
19,Padang,ID,77.23,overcast clouds,-0.9492,100.3543,
25,Atuona,PF,79.20,light rain,-9.8000,-139.0333,
27,Hilo,US,79.79,light rain,19.7297,-155.0900,
31,Victoria,HK,82.45,overcast clouds,22.2855,114.1577,
35,Korla,CN,78.73,scattered clouds,41.7597,86.1469,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#The rows with an empyt string weren't showing up in a isnull check, so I put this here to check if dropna would make the desired changes
clean_hotels = hotel_df.dropna()
print(len(clean_hotels), len(hotel_df))

204 204


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
print(len(clean_hotels), len(hotel_df), len(clean_hotel_df))
#last line to prove that the desired affect was achieved, given the failure in the previous cell

204 204 191


In [11]:
# 8a. Create the output File (CSV)
#clean_hotel_df.head()
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))